In [1]:
%matplotlib inline
from matplotlib import pyplot as pt
import pandas as pd

import os
import glob

/home/ian/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def load_local_data():
    csv_path = os.path.join(os.getcwd(), 'data/*.txt')
    all_files = glob.glob(csv_path)
    
    print 'reading data from files:'
    for f in all_files:
        print f
    
    data = pd.concat((pd.read_csv(f) for f in all_files))
    
    # strip whitespace off column headers (eg 'EXITS               ' has trailing spaces)
    data = data.rename(columns=lambda x: x.strip())
    return data

data = load_local_data()
print "\ngot %i rows total." % len(data)
data.head()

reading data from files:
/home/ian/metis/team_benson/data/turnstile_160402.txt
/home/ian/metis/team_benson/data/turnstile_160326.txt

got 387100 rows total.


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282


In [3]:
# How many stations are there?
unique_stations = data.STATION.unique()
print len(unique_stations)

373


In [4]:
# make a datetime column
data['DATETIME'] = pd.to_datetime(data.DATE + ' ' + data.TIME, format="%m/%d/%Y %H:%M:%S")
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00


In [5]:
# Calclate new columns ENTRIES_DIFF and EXITS_DIFF for each unique turnstile/device.
# The first entry is NaN, the rest of the entries are the difference from the previous.
turnstile_groupby = data.groupby(['C/A','UNIT','SCP','STATION'])
for diff_field in ('ENTRIES','EXITS'):
    data[diff_field + '_DIFF'] = turnstile_groupby[diff_field].transform(pd.Series.diff)

In [6]:
# sanity check for getting the differences:
nan_entries = len(data[data.ENTRIES_DIFF.isnull()])
total_unique_turnstiles = len(turnstile_groupby)
print 'the number of NaN entries should equal the total unique turnstiles.'
print nan_entries, total_unique_turnstiles

assert nan_entries == total_unique_turnstiles

the number of NaN entries should equal the total unique turnstiles.
4595 4595


# TODO
Groupby station and datetime, then sum.

In [20]:
# group by station name and datetime,
station_time_groupby = data.groupby(['STATION','DATETIME'])

# data.join(station_time_groupby['ENTRIES_DIFF','EXITS_DIFF'].sum(), on=['STATION','DATETIME'], rsuffix='_r')
station_time_groupby['ENTRIES_DIFF','EXITS_DIFF'].sum().head(100)

# clean_entries = station_time_groupby['ENTRIES_DIFF'].sum()
# clean_exits = station_time_groupby['EXITS_DIFF'].sum()

# clean_data = data.drop(['ENTRIES_DIFF','EXITS_DIFF'], axis=1)
# clean_data['ENTRIES'] = clean_entries
# clean_data['EXITS'] = clean_exits

# clean_data.head(20)

ENTRIES_DIFF  EXITS_DIFF
STATION DATETIME                                     
1 AV    2016-03-19 00:00:00       -302959     -189288
        2016-03-19 04:00:00          1909         801
        2016-03-19 08:00:00           570         777
        2016-03-19 12:00:00          2488        1972
        2016-03-19 16:00:00          4229        2511
        2016-03-19 20:00:00          4453        2821
        2016-03-20 00:00:00          3830        2261
        2016-03-20 04:00:00          1680         914
        2016-03-20 08:00:00           442         486
        2016-03-20 12:00:00          1810        1300
        2016-03-20 16:00:00          3079        2190
        2016-03-20 20:00:00          3145        2364
        2016-03-21 00:00:00          2252        1220
        2016-03-21 04:00:00           407         180
        2016-03-21 08:00:00          1854        2555
        2016-03-21 12:00:00          5488        3916
        2016-03-21 16:00:00          4964        2566
        2016-03-21 20:00:00          7053        3522
        2016-03-22 00:00:00          3415        1668
        2016-03-22 04:00:00           527         247
        2016-03-22 08:00:00          1895        2635
        2016-03-22 12:00:00          5602        4148
        2016-03-22 16:00:00          5051        2654
        2016-03-22 20:00:00          7420        3417
        2016-03-23 00:00:00          3924        1830
        2016-03-23 04:00:00           682         303
        2016-03-23 08:00:00          1781        2576
        2016-03-23 12:00:00          5654        4057
        2016-03-23 16:00:00          5158        2531
        2016-03-23 20:00:00          7489        3560
...                                   ...         ...
        2016-03-30 16:00:00          5172        2545
        2016-03-30 20:00:00          7609        3658
        2016-03-31 00:00:00          4115        2073
        2016-03-31 04:00:00           732         321
        2016-03-31 08:00:00          1862        2612
        2016-03-31 09:25:33           451        1068
        2016-03-31 12:00:00          5136        3127
        2016-03-31 16:00:00          5172        2699
        2016-03-31 20:00:00          7403        3552
        2016-04-01 00:00:00          4725        2205
        2016-04-01 04:00:00           898         370
        2016-04-01 08:00:00          1696        2515
        2016-04-01 12:00:00          5676        4197
        2016-04-01 16:00:00          5483        2668
        2016-04-01 20:00:00          7495        3680
103 ST  2016-03-19 01:00:00       -359330     -185652
        2016-03-19 05:00:00           183         354
        2016-03-19 09:00:00          2108        1039
        2016-03-19 13:00:00          5303        1835
        2016-03-19 17:00:00          5603        2540
        2016-03-19 21:00:00          4445        2662
        2016-03-20 01:00:00          1946        1904
        2016-03-20 05:00:00           238         375
        2016-03-20 09:00:00          1503         615
        2016-03-20 13:00:00          4545        1550
        2016-03-20 17:00:00          4634        2306
        2016-03-20 21:00:00          3091        2449
        2016-03-21 01:00:00          1175        1282
        2016-03-21 05:00:00           186         159
        2016-03-21 09:00:00          8211        2587

[100 rows x 2 columns]

In [37]:
# let's try selecting all the rows that match the 10th row's STATION and DATETIME
# Remember, 0th row is all NaN!!!
ST, DT = data.iloc[10][['STATION','DATETIME']]
tenth_row_station_dt = data[(data['STATION'] == ST) & (data['DATETIME'] == DT)]
tenth_row_station_dt
# tenth_row_station_dt.groupby(['STATION']).sum()


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRIES_DIFF,EXITS_DIFF
10,A002,R051,02-00-00,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,5595746,1893282,2016-03-27 16:00:00,0,0
53,A002,R051,02-00-01,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,5143982,1127506,2016-03-27 16:00:00,0,0
96,A002,R051,02-03-00,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,671757,2532310,2016-03-27 16:00:00,0,0
139,A002,R051,02-03-01,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,5116889,8091742,2016-03-27 16:00:00,0,0
182,A002,R051,02-03-02,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,4844390,6685484,2016-03-27 16:00:00,0,0
225,A002,R051,02-03-03,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,4474371,5398459,2016-03-27 16:00:00,0,0
268,A002,R051,02-03-04,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,5673113,3116021,2016-03-27 16:00:00,0,1
311,A002,R051,02-03-05,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,9170413,1251248,2016-03-27 16:00:00,1,0
354,A002,R051,02-03-06,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,7195572,471294,2016-03-27 16:00:00,0,0
397,A002,R051,02-05-00,59 ST,NQR456,BMT,03/27/2016,16:00:00,REGULAR,1268,0,2016-03-27 16:00:00,2,0
